In [1]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///example.db', echo=True)

### ORM 초기화

In [2]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

session_maker = sessionmaker(bind=engine)
session = session_maker()

Base = declarative_base()

### 테이블 정의

In [3]:
from sqlalchemy import Column, Integer, String

class Demo(Base):
    __tablename__ = 'demos'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    password = Column(String)

### metadata확인
* demos 테이블은 engine이 메타데이터로 관리
* 아직 databse에 미반영

In [4]:
for k,v in Base.metadata.tables.items():
    print(f"{k}: {v}")

demos: demos


In [5]:
Base.metadata.tables

FacadeDict({'demos': Table('demos', MetaData(), Column('id', Integer(), table=<demos>, primary_key=True, nullable=False), Column('name', String(), table=<demos>), Column('password', String(), table=<demos>), schema=None)})

### metadata -> 데이터베이스 반영
* engine은 commit을 실행하여 데이터베이스에 테이블을 반영(실행 결과 맨 마지막 줄에 commit로그)

In [6]:
Base.metadata.create_all(engine)

2021-08-14 16:57:06,328 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-14 16:57:06,328 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("demos")
2021-08-14 16:57:06,329 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-14 16:57:06,330 INFO sqlalchemy.engine.Engine COMMIT


### 세션에 객체 추가
* 세션에만 객체가 저장될 뿐 아직 데이터베이스에 미반영

In [7]:
# 객체 생성
demo = Demo(name="hello", password="world")

session.add(demo)

In [14]:
# 생성한 객체 조회
find_demos = session.query(Demo).filter_by(name="hello").all()

for demo in find_demos:
    print(demo.name)

2021-08-14 16:58:24,950 INFO sqlalchemy.engine.Engine SELECT demos.id AS demos_id, demos.name AS demos_name, demos.password AS demos_password 
FROM demos 
WHERE demos.name = ?
2021-08-14 16:58:24,952 INFO sqlalchemy.engine.Engine [cached since 47.1s ago] ('hello',)
hello


### 세션 commit
* 세션에 저장된 데이터를 데이터베이스에 commit

In [15]:
session.commit()

2021-08-14 16:59:22,649 INFO sqlalchemy.engine.Engine COMMIT
